In [1]:
from tensorflow.python.keras.layers import Input,Dense,Reshape,Lambda,RNN,Concatenate,StackedRNNCells
from tensorflow.python.keras.models import Model,load_model
from tensorflow.python.keras.utils import multi_gpu_model
from tensorflow.python.keras.optimizers import Adam,SGD
from tensorflow.python.keras.callbacks import LearningRateScheduler,Callback,ModelCheckpoint,TensorBoard

import tensorflow as tf
import datetime
import os
import numpy as np
import pickle

%env PYTHONHASHSEED=0
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorflow/

env: PYTHONHASHSEED=0


/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ai/anaconda3/envs/gpu/lib/python3.7/site

In [2]:
from sklearn.model_selection import train_test_split
#features
n2v_start = np.load("start_emb_128.npy")
n2v_end = np.load("end_emb_128.npy")
lanes = np.load("lanes.npy")
oneway = np.load("oneway.npy")
roadtype = np.load("highway.npy")
maxspeed = np.load("maxspeed.npy")
nodetype = np.load("nodetype.npy")
lanes_df = np.load("lanes_df.npy")
#target
meanspeed = np.load("meanspeed_only.npy")

#split
n2v_start_train, n2v_start_test, n2v_end_train, n2v_end_test, lanes_train, lanes_test, oneway_train, oneway_test, roadtype_train, roadtype_test, maxspeed_train, maxspeed_test, nodetype_train, nodetype_test, lanes_df_train, lanes_df_test, meanspeed_train, meanspeed_test = train_test_split(n2v_start, n2v_end, lanes,oneway,roadtype,maxspeed,nodetype,lanes_df,meanspeed)
del n2v_start
del n2v_end
del lanes
del oneway
del roadtype
del maxspeed
del nodetype
del lanes_df
del meanspeed

In [6]:
embinplist = list()
n2vlist = list()
inplist = list()

n2v_start_inp = Input((128,))
n2v_start_reg = Dense(128,activation = "relu", use_bias = True)(n2v_start_inp)
inplist.append(n2v_start_inp)
n2vlist.append(n2v_start_reg)

n2v_end_inp = Input((128,))
n2v_end_reg = Dense(128,activation = "relu", use_bias = True)(n2v_end_inp)
inplist.append(n2v_end_inp)
n2vlist.append(n2v_end_reg)

lanes_inp = Input((1,))
lanes_reg= Dense(1,activation = "relu", use_bias = True)(lanes_inp)
inplist.append(lanes_inp)
embinplist.append(lanes_reg)

oneway_inp = Input((2,))
oneway_reg= Dense(2,activation = "relu", use_bias = True)(oneway_inp)
inplist.append(oneway_inp)
embinplist.append(oneway_reg)

roadtype_inp = Input((13,))
roadtype_reg= Dense(8,activation = "relu", use_bias = True)(roadtype_inp)
inplist.append(roadtype_inp)
embinplist.append(roadtype_reg)

maxspeed_inp = Input((1,))
maxspeed_reg= Dense(1,activation = "relu", use_bias = True)(maxspeed_inp)
inplist.append(maxspeed_inp)
embinplist.append(maxspeed_reg)

nodetype_inp = Input((11,))
nodetype_reg= Dense(8,activation = "relu", use_bias = True)(nodetype_inp)
inplist.append(nodetype_inp)
embinplist.append(nodetype_reg)

lanes_df_inp = Input((1,))
lanes_df_reg= Dense(1,activation = "relu", use_bias = True)(lanes_df_inp)
inplist.append(lanes_df_inp)
embinplist.append(lanes_df_reg)

n2v = Concatenate()(n2vlist)
n2v_reg = Dense(64,activation = "relu",use_bias = True)(n2v)
embinplist.append(n2v_reg)

combine = Concatenate()(embinplist)
proj = Dense(32,activation = "relu",use_bias = True)(combine)
proj = Dense(1,activation = "linear")(proj)

emb = Model(inplist,proj)

In [22]:
emb.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_29 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_30 (InputLayer)           [(None, 128)]        0                                            
__________________________________________________________________________________________________
dense_40 (Dense)                (None, 64)           8256        input_29[0][0]                   
__________________________________________________________________________________________________
dense_41 (Dense)                (None, 64)           8256        input_30[0][0]                   
____________________________________________________________________________________________

In [7]:
adam=Adam(lr=0.01,beta_1=0.9,beta_2=0.999,amsgrad=True)
emb.compile(optimizer=adam, loss='mae',metrics=['mae'])

In [8]:
history =emb.fit([n2v_start_train,n2v_end_train,lanes_train,oneway_train,roadtype_train,maxspeed_train,nodetype_train,lanes_df_train],
        meanspeed_train,
        epochs=100,batch_size=256,
        validation_data=([n2v_start_test,n2v_end_test,lanes_test,oneway_test,roadtype_test,maxspeed_test,nodetype_test,lanes_df_test],
        meanspeed_test)
       )

Train on 86992 samples, validate on 28998 samples
Epoch 1/100
86992/86992 [==============================] - 4s 44us/sample - loss: 8.2605 - mean_absolute_error: 8.2605 - val_loss: 6.8024 - val_mean_absolute_error: 6.8024
Epoch 2/100
86992/86992 [==============================] - 3s 39us/sample - loss: 6.1792 - mean_absolute_error: 6.1792 - val_loss: 5.7845 - val_mean_absolute_error: 5.7845
Epoch 3/100
86992/86992 [==============================] - 3s 39us/sample - loss: 5.2870 - mean_absolute_error: 5.2870 - val_loss: 5.6476 - val_mean_absolute_error: 5.6476
Epoch 4/100
86992/86992 [==============================] - 3s 39us/sample - loss: 4.8256 - mean_absolute_error: 4.8256 - val_loss: 5.0766 - val_mean_absolute_error: 5.0766
Epoch 5/100
86992/86992 [==============================] - 3s 39us/sample - loss: 4.4493 - mean_absolute_error: 4.4493 - val_loss: 4.8187 - val_mean_absolute_error: 4.8187
Epoch 6/100
86992/86992 [==============================] - 3s 39us/sample - loss: 4.2275 -

86992/86992 [==============================] - 3s 37us/sample - loss: 2.0744 - mean_absolute_error: 2.0744 - val_loss: 3.7969 - val_mean_absolute_error: 3.7969
Epoch 96/100
86992/86992 [==============================] - 3s 39us/sample - loss: 2.0758 - mean_absolute_error: 2.0758 - val_loss: 3.7800 - val_mean_absolute_error: 3.7800
Epoch 97/100
86992/86992 [==============================] - 3s 38us/sample - loss: 2.0682 - mean_absolute_error: 2.0682 - val_loss: 3.7834 - val_mean_absolute_error: 3.7834
Epoch 98/100
86992/86992 [==============================] - 3s 39us/sample - loss: 2.0607 - mean_absolute_error: 2.0607 - val_loss: 3.8183 - val_mean_absolute_error: 3.8183
Epoch 99/100
86992/86992 [==============================] - 3s 39us/sample - loss: 2.0656 - mean_absolute_error: 2.0656 - val_loss: 3.8698 - val_mean_absolute_error: 3.8698
Epoch 100/100
86992/86992 [==============================] - 3s 39us/sample - loss: 2.0641 - mean_absolute_error: 2.0641 - val_loss: 3.7929 - val_me

In [49]:
emb.save('/home/fyp2/Ideaer/Madrid/model/all')

In [9]:
loss_hist = history.history['loss']
mae_history = history.history["val_mean_absolute_error"]
print("min loss :"+str(np.min(loss_hist)))
print("min val mae :"+str(np.min(mae_history)))

min loss :2.060740846997349
min val mae :3.75692


## node2vec 256 --> 32

In [8]:
loss_hist = history.history['loss']
mae_history = history.history["val_mean_absolute_error"]
print("min loss :"+str(np.min(loss_hist)))
print("min val mae :"+str(np.min(mae_history)))

min loss :2.109027302043719
min val mae :3.7186508


## node2vec 256 -->128 --> 64

In [13]:
loss_hist = history.history['loss']
mae_history = history.history["val_mean_absolute_error"]
print("min loss :"+str(np.min(loss_hist)))
print("min val mae :"+str(np.min(mae_history)))

min loss :2.819155234502961
min val mae :4.35377


## node2vec 256 -->128 --> 32

In [17]:
loss_hist = history.history['loss']
mae_history = history.history["val_mean_absolute_error"]
print("min loss :"+str(np.min(loss_hist)))
print("min val mae :"+str(np.min(mae_history)))

min loss :3.0678007390452193
min val mae :4.451634
